# Import Libraries

In [1]:
import pandas as pd
import os

# Import Datasets

In [2]:
os.chdir('/content/drive/MyDrive/Colab/Datasets/fraud_detection_sets')

# Load datasets
df_transactions = pd.read_csv('transactions_data.csv')
df_users = pd.read_csv('users_data.csv')
df_cards = pd.read_csv('cards_data.csv')
df_fraud_labels = pd.read_json('train_fraud_labels.json').reset_index() # Use read_json to read json file, index the records

# Function to display dataset details
def display_df_info(df, name):
    print(f"\n{name} - Info:")
    print(df.info())

# Display info for each dataframe
display_df_info(df_transactions, "Transactions")
display_df_info(df_users, "Users")
display_df_info(df_cards, "Cards")
display_df_info(df_fraud_labels, "Fraud Labels")


Transactions - Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2661183 entries, 0 to 2661182
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(1), int64(5), object(6)
memory usage: 243.6+ MB
None

Users - Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000

# Data Preprocessing

## Check for NAN & Duplicates values

In [3]:
# Check for missing values in all datasets
def check_missing_values(df):
    return df.isnull().sum()

# Print sum of missing values for each set
print(f"\n{check_missing_values(df_transactions)}")
print(f"\n{check_missing_values(df_users)}")
print(f"\n{check_missing_values(df_cards)}")
print(f"\n{check_missing_values(df_fraud_labels)}")

# Drop duplicates for all datasets if any
df_transactions.drop_duplicates(inplace=True)
df_users.drop_duplicates(inplace=True)
df_cards.drop_duplicates(inplace=True)
df_fraud_labels.drop_duplicates(inplace=True)


id                      0
date                    0
client_id               0
card_id                 0
amount                  0
use_chip                0
merchant_id             0
merchant_city           0
merchant_state     312815
zip                330584
mcc                     0
errors            2618999
dtype: int64

id                   0
current_age          0
retirement_age       0
birth_year           0
birth_month          0
gender               0
address              0
latitude             0
longitude            0
per_capita_income    0
yearly_income        0
total_debt           0
credit_score         0
num_credit_cards     0
dtype: int64

id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_

## Feature Selections

In [4]:
# Drop merchant_state, zip and errors column on transactions df
df_transactions.drop(['merchant_state', 'zip', 'errors'], axis=1, inplace=True)

# Drop birth_year, birth_month retirement_age, addresss, lat, long and total_debt columns on users df
df_users.drop(['birth_year', 'birth_month', 'retirement_age', 'address', 'latitude', 'longitude', 'total_debt'], axis=1, inplace=True)

# Drop card_number, expires cvv, id and acct_open_date columns on cards df
df_cards.drop(['card_number', 'expires', 'cvv', 'acct_open_date', 'id'], axis=1, inplace=True)

## Data Cleaning

In [5]:
# Convert amount column from string to numeric on transactions df
df_transactions['amount'] = df_transactions['amount'].str.lstrip('$').astype(float)

# Convert per_capita_income and yearly_income column from string to numeric on users df
df_users['per_capita_income'] = df_users['per_capita_income'].str.lstrip('$').astype(float)
df_users['yearly_income'] = df_users['yearly_income'].str.lstrip('$').astype(float)

# # Convert credit_limit column from string to numeric on cards df
df_cards['credit_limit'] = df_cards['credit_limit'].str.lstrip('$').astype(float)

# Rename columns
df_users.rename(columns={'id': 'client_id'}, inplace=True) # Rename id to client_id
df_fraud_labels.rename(columns={'index': 'transaction_id', 'target': 'is_fraud'}, inplace=True) # Rename index to transaction_id & target to is_fraud
df_transactions.rename(columns={'id': 'transaction_id'}, inplace=True) # Rename id to transaction_id

## Merging Datasets & One-hot Encoding

In [6]:
# Merge Users to transaction
df_transactions = pd.merge(df_transactions, df_users, on="client_id")

# Merge Cards to transaction
df_transactions = pd.merge(df_transactions, df_cards, on='client_id')

# Merge Fraud Labels to transaction
df_transactions = pd.merge(df_transactions, df_fraud_labels, on='transaction_id')

# Run One-hot encoding to categorical columns
df_transactions_encoded = pd.get_dummies(df_transactions, columns=["use_chip", "gender", "card_brand", "card_type", "has_chip", "card_on_dark_web", "is_fraud"], drop_first=True)

,transaction_id,date,client_id,card_id,amount,merchant_id,merchant_city,mcc,current_age,per_capita_income,...,use_chip_Online Transaction,use_chip_Swipe Transaction,gender_Male,card_brand_Discover,card_brand_Mastercard,card_brand_Visa,card_type_Debit,card_type_Debit (Prepaid),has_chip_YES,is_fraud_Yes
0,7565784,2010-01-24 09:24:00,1645,1047,3.39,22938,Cleveland,5812,62,29692.0,...,False,True,False,False,True,False,False,True,False,False
1,7565784,2010-01-24 09:24:00,1645,1047,3.39,22938,Cleveland,5812,62,29692.0,...,False,True,False,False,True,False,True,False,False,False
2,7565784,2010-01-24 09:24:00,1645,1047,3.39,22938,Cleveland,5812,62,29692.0,...,False,True,False,False,False,True,True,False,False,False
3,7571127,2010-01-25 13:52:00,1448,5907,75.62,98374,Landrum,5411,48,18899.0,...,False,True,False,False,True,False,False,False,True,False
4,7571127,2010-01-25 13:52:00,1448,5907,75.62,98374,Landrum,5411,48,18899.0,...,False,True,False,False,False,True,True,False,True,False


# Data Modeling

In [8]:
df_transactions_encoded.isnull().sum()

,0
transaction_id,0
date,0
client_id,0
card_id,0
amount,0
merchant_id,0
merchant_city,0
mcc,0
current_age,0
per_capita_income,0
